In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score

import matplotlib as plt
import tensorflow as tf

In [2]:
# matplotlib
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')


# Pandas and Numpy
import numpy as np
import pandas as pd

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import psycopg2

In [3]:
#import connection string (youll need to create this)
from config import conn

# create the engine
engine = create_engine(conn)
Base = automap_base()

# reflect an existing database into a new model
Base.prepare(engine, reflect=True)

# reflect the tables
#Recession_Indicator = Base.classes.recession_indicator_by_day
Recession_Indicator = Base.classes.us_recession
Historical_Price = Base.classes.historical_price_by_day

In [4]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [5]:
#create df from Historical_Price table

results = []

results = session.query(Historical_Price.date, \
                        Historical_Price.open, \
                        Historical_Price.high, \
                        Historical_Price.low, \
                        Historical_Price.close, \
                        Historical_Price.ticker)


price_df = pd.DataFrame(results).set_index(['date','ticker'])
price_df.head()

,,open,high,low,close
date,ticker,,,,
36643,SPY,$143.00,$147.34,$143.00,$146.00
36644,SPY,$147.00,$147.86,$145.06,$145.09
36647,SPY,$146.56,$148.48,$145.84,$147.06
36648,SPY,$145.50,$147.13,$144.13,$144.13
36649,SPY,$144.00,$144.00,$139.78,$140.75


In [13]:
#create df from Recession_Indicator table
results = []

results = session.query(Recession_Indicator.date, \
                        Recession_Indicator.recession_indicator, \
                        Recession_Indicator.country)


#recession_df = pd.DataFrame(results).set_index(['country'])
recession_df = pd.DataFrame(results)

recession_df.head()

,date,recession_indicator,country
0,25569,True,us
1,25570,True,us
2,25571,True,us
3,25572,True,us
4,25573,True,us


In [14]:

# create the engine
engine = create_engine(conn)
Base = automap_base()

# reflect an existing database into a new model
Base.prepare(engine, reflect=True)

# reflect the tables
#Recession_Indicator = Base.classes.recession_indicator_by_day
Recession_Indicator = Base.classes.us_recession
Historical_Price = Base.classes.historical_price_by_day

In [15]:
X = recession_df.copy()
X = X.drop("recession_indicator", axis=1)
X = pd.get_dummies(X, columns=["country"])
X.head()

,date,country_us
0,25569,1
1,25570,1
2,25571,1
3,25572,1
4,25573,1


In [16]:
y_cols = recession_df["recession_indicator"] 

y = pd.DataFrame()

y["recession_indicator"] = y_cols
y.head()

,recession_indicator
0,True
1,True
2,True
3,True
4,True


In [17]:
y['recession_indicator'].value_counts()

False    9712
True     8550
Name: recession_indicator, dtype: int64

In [18]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_cols, random_state=1)

In [19]:
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs", max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")



 Logistic regression model accuracy: 0.538
